In [0]:
# mounting with dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# importing libraries
from __future__ import print_function
from __future__ import division

import os,cv2,scipy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import torchvision.transforms.functional as TF
from PIL import Image
import matplotlib.pyplot as plt
from scipy import io as sio
from scipy import ndimage, misc
import csv
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [0]:
data_dir = "/content/drive/My Drive/HW3_data"

#setting alexnet model 
model_name = "alexnet"

# Number of classes in the dataset
num_classes = 8

# Batch size for training
batch_size = 8

# Number of training epochs
num_epochs = 15

# the last layer parameters are updated, the others remain fixed.
extract_features = True

In [0]:
# Reading Images
def readImages(imagesPath, imagesCount):
  images = []
  
  #Looping through all the images 
  for i in range(imagesCount):
    #Reading image paths
    path = imagesPath + '/' + str(i+1) + '.jpg'
    
    #reading the image in the dataset from path
    img = scipy.misc.imread(path)
    
    #appending all the images in the images array
    images.append(img)
    
  #converting it to a numpy array
  images = np.array(images)
  
  #Image Dimensions
  print(images.shape)
  return images

In [0]:
# storing all the labels in labels variable
def readLabels(labelsPath):
  
  #opening the csv file
	with open(labelsPath,'rU') as csvfile:
    #reading the csv file and converting to list
		csvfile = csv.reader(csvfile, delimiter=',')
		csvdata = list(csvfile)
    
    #saving the labels in labels variable in integer format
		labels = map(int, csvdata[0])
    
    #returning the list
		return list(labels)

In [0]:
# train image path
training_images_path = '/content/drive/My Drive/HW3_data/train'

# reading images in train dataset
train_images = readImages(training_images_path, 1888)

# dimensions of train_images variable
train_images.shape

# path of train data labels
training_labels_path = '/content/drive/My Drive/HW3_data/train_labels.csv'

# saving the labels in train_labels
train_labels = readLabels(training_labels_path)



# test images path in the dataset
testing_images_path = '/content/drive/My Drive/HW3_data/test'

# reading images in test dataset
test_images = readImages(testing_images_path, 800)

# dimensions of test_images variable
test_images.shape

# path of test data labels
testing_labels_path = '/content/drive/My Drive/HW3_data/test_labels.csv'

# saving the labels in test_labels
test_labels = readLabels(testing_labels_path)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.


(1888, 256, 256, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.


(800, 256, 256, 3)


In [0]:
# we want all of the other parameters to not require gradient
# therefore setting the parameter false
def set_parameter_requires_grad(model, feature_extracting):

    #feature_extracting is true
    if feature_extracting:
        for param in model.parameters():
            #By default, when we load a pretrained model all of the parameters have gradients. That is we have param.requires_grad=True
            param.requires_grad = False

In [0]:
# pretraining the alexnet model
alexnet = models.alexnet(pretrained=True)

In [0]:
# Alexnet Achitecture
alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [0]:
# initializing the Alexnet model
def initialize_model(model_name, num_classes, extract_features, use_pretrained=True):
    input_size = 0
    model_features = None
    
    if model_name == "alexnet":
        #alexnet is trained with imagenet dataset. 
        model_features = models.alexnet(pretrained=use_pretrained)
        
        #removing gradients so that we can use for feature extraction
        set_parameter_requires_grad(model_features, extract_features)
        num_ftrs = model_features.classifier[6].in_features
  
        #we print the model architecture in such a way that we see the model output comes from the 6th layer of the classifier
        # output layer has 8 layers because we have 8 classes
        model_features.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224


    return model_features, input_size

  

In [0]:
# saving training labels data 
train_labels = np.array(pd.read_csv('/content/drive/My Drive/HW3_data/train_labels.csv',header =None))[0]
# saving testing labels data
test_labels = np.array(pd.read_csv('/content/drive/My Drive/HW3_data/test_labels.csv',header =None))[0]

In [0]:
# Face Landmarks Dataset
class FaceLandmarksDataset(Dataset):

    def __init__(self,root_dir,total_count, csv_file, transform=None):
              #csv_file (string): Path to the csv file with annotations.
              #root_dir (string): Directory with all the images.
              #transform (callable, optional): Optional transform to be applied on a sample.
        
        self.transform = transform
        self.csv_file = np.array(pd.read_csv(csv_file,header=None))[0]
        self.root_dir = root_dir
        self.total_count = total_count
    
    def __len__(self):
        return self.total_count

    def __getitem__(self, image_no):
        #joining the path
        img_name = os.path.join(self.root_dir,
                                str(image_no+1)+".jpg")
        image = Image.open(img_name)
        
        #creating dictionary with image's corresponding label
        image_label_dict = {
            'image': image,
            'label': self.csv_file[image_no]-1
           }

        if self.transform:
            image_label_dict['image'] = self.transform(image_label_dict['image'])

        return image_label_dict

In [0]:
# Load Data
# Data augmentation and normalization for training
trainDataset = FaceLandmarksDataset('/content/drive/My Drive/HW3_data/train',1888,'/content/drive/My Drive/HW3_data/train_labels.csv',transforms.Compose([

        #resizing
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),

        #The images have to be loaded in to a range of [0, 1] and then normalized
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

# Just normalization for validation
testDataset = FaceLandmarksDataset('/content/drive/My Drive/HW3_data/test',800,'/content/drive/My Drive/HW3_data/test_labels.csv',transforms.Compose([
        
        #resizing
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        
        #The images have to be loaded in to a range of [0, 1] and then normalized
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

In [0]:
# Initialize the model for this run
model_features, input_size = initialize_model("alexnet", 8, extract_features, use_pretrained=True)

In [0]:
# Initializing Datasets and Dataloaders as inputs for model
# Create training and validation dataloaders
trainDataloader = torch.utils.data.DataLoader(trainDataset,batch_size = 8, shuffle = True, num_workers=4)
testDataloader = torch.utils.data.DataLoader(testDataset,batch_size = 8, shuffle = True, num_workers=4)
Dataloaders_dict = {trainDataloader,testDataloader}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
# Send the model to GPU
model_features = model_features.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.

#Update intialized parameters
params_to_update = model_features.parameters()
print("Params to learn:")

if extract_features:
    #parameters to be updated in this run 
    params_to_update = []
    
    #iterating through all the parameters
    for name,param in model_features.named_parameters():
        #check if gradient is true, if true then update and set to false
        if param.requires_grad == True:
            #saving all the parameters to be updated to params_to_update
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_features.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# All parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [0]:
# Selecting Cross Entropy as the loss function
criterion = nn.CrossEntropyLoss()

In [0]:
# loop over the dataset multiple times
for epoch in range(num_epochs):  
# initializing with 0
    running_loss = 0.0
    for i, data in enumerate(trainDataloader, 0):
        #get the inputs
        inputs, labels = data['image'],data['label']
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        #zero the parameter gradients
        optimizer_ft.zero_grad()
        outputs = model_features(inputs)
        
        #calculating loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_ft.step()

        #print statistics
        running_loss += loss.item()
        
        #print every 20 mini-batches
        if i % 20 == 19:    
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 1.631
[1,    20] loss: 0.765
[1,    30] loss: 0.513
[1,    40] loss: 0.473
[1,    50] loss: 0.713
[1,    60] loss: 0.413
[1,    70] loss: 0.584
[1,    80] loss: 0.351
[1,    90] loss: 0.543
[1,   100] loss: 0.420
[1,   110] loss: 0.509
[1,   120] loss: 0.432
[1,   130] loss: 0.356
[1,   140] loss: 0.416
[1,   150] loss: 0.414
[1,   160] loss: 0.602
[1,   170] loss: 0.337
[1,   180] loss: 0.473
[1,   190] loss: 0.555
[1,   200] loss: 0.599
[1,   210] loss: 0.684
[1,   220] loss: 0.429
[1,   230] loss: 0.489
[2,    10] loss: 0.470
[2,    20] loss: 0.434
[2,    30] loss: 0.316
[2,    40] loss: 0.702
[2,    50] loss: 0.219
[2,    60] loss: 0.266
[2,    70] loss: 0.372
[2,    80] loss: 0.678
[2,    90] loss: 0.322
[2,   100] loss: 0.257
[2,   110] loss: 0.408
[2,   120] loss: 0.373
[2,   130] loss: 0.628
[2,   140] loss: 0.294
[2,   150] loss: 0.337
[2,   160] loss: 0.330
[2,   170] loss: 0.517
[2,   180] loss: 0.432
[2,   190] loss: 0.212
[2,   200] loss: 0.577
[2,   210] 

In [0]:
# evaluate the model on test data
correct = 0
total = 0

with torch.no_grad():
    #iterating through test data
    for i, data in enumerate(testDataloader, 0):
        
        # get the inputs
        inputs, labels = data['image'],data['label']
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_features(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        
        #calculating accuracy
        correct += (predicted == labels).sum().item()
        
print('Network Accuracy on test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 800 test images: 91 %
